<a href="https://colab.research.google.com/github/MPrazzoli/financial_markets_academy/blob/master/_8_jupyter_notebooks/ipynb_notebook/LogisticReg/Mirko/0_datascienceClassificationLogistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# includere altri indici come il moving average rsi e mm, RSI, alpha beta il knn (cerca cluster e costruisci indici fittizi) o il k means, cerca altri dati di mercato, futures e indici metti altre stock appartenenti allo stesso settore,
# prezzo del petrolio, il cambio, la curva dei tassi (orizzontalmente per scadenze) 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# python_version 3.7.10

### &nbsp;&nbsp;&nbsp; %CLASS% StockClass: used to create the stock dictionary

In [1]:
class StockClass(object):

    # Initialization of the StockClass object with the ticker symbol which is use to construct a yf.Ticker object
    def __init__(self, ticker, isin=None, exchangeid=None, sector=None, industry=None, country=None, pe=None, eps=None,
                 insiderown=None, shsout=None, shsfloat=None, mktcap=None, income=None, sales=None,
                 booksh=None, pb=None, roa=None, tp=None, roe=None, roi=None, employees=None, debteq=None, 
                 groupby=None, confusion_matrix=None, accuracy_report=None, 
                 confusion_matrix2=None, accuracy_report2=None, lr=None, lr2=None):
        self.name = ticker
        self.isin = isin
        self.exchangeid = exchangeid
        # self.history is the method of the SotckClass object to store data in DataFrame format
        self.history = pd.DataFrame
        self.investing = pd.DataFrame
        self.pickle = pd.DataFrame
        self.not_found = np.array([['date', 'ticker']])
        self.nanDiv = False
        self.nanSplit = False
        self.sector = None
        self.industry = industry
        self.country = country
        self.pe = pe
        self.eps = eps
        self.insiderown = insiderown
        self.shsout = shsout
        self.shsfloat = shsfloat
        self.mktcap = mktcap
        self.income = income
        self.sales = sales
        self.bookh = booksh
        self.pb = pb
        self.roa = roa
        self.tp = tp
        self.roe = roe
        self.roi = roi
        self.employees = employees
        self.debteq = debteq
        self.rsi = pd.DataFrame
        self.groupby = groupby
        self.confusion_matrix = confusion_matrix
        self.accuracy_report = accuracy_report
        self.logistic_regression = lr
        self.confusion_matrix2 = confusion_matrix2
        self.accuracy_report2 = accuracy_report2
        self.logistic_regression2 = lr2

# 1. First part: import data, cleaning and arranging 

### &nbsp;&nbsp;&nbsp; * Main packages import

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### &nbsp;&nbsp;&nbsp; * Data import

In [3]:
open_df_original = pd.read_csv('/content/drive/MyDrive/Data/20210322/open', index_col='date')
high_df_original = pd.read_csv('/content/drive/MyDrive/Data/20210322/high', index_col='date')
low_df_original = pd.read_csv('/content/drive/MyDrive/Data/20210322/low', index_col='date')
adjclose_df_original = pd.read_csv('/content/drive/MyDrive/Data/20210322/adjclose', index_col='date')
volume_df_original = pd.read_csv('/content/drive/MyDrive/Data/20210322/volume', index_col='date')

In [4]:
# to drop columns with the end of the series NaN (probably not quoted anymore)
if open_df_original.isnull().values.any():
    print('open shape before: ', open_df_original.shape)
    open_df_original.dropna(axis=1, how='any', inplace=True)
    print('open shape after: ', open_df_original.shape)    
if high_df_original.isnull().values.any():
    print('high shape before: ', high_df_original.shape)
    high_df_original.dropna(axis=1, how='any', inplace=True)
    print('high shape after: ', high_df_original.shape)  
if low_df_original.isnull().values.any():
    print('low shape before: ', low_df_original.shape)
    low_df_original.dropna(axis=1, how='any', inplace=True)
    print('low shape after: ', low_df_original.shape)    
if adjclose_df_original.isnull().values.any():
    print('high shape before: ', adjclose_df_original.shape)
    adjclose_df_original.dropna(axis=1, how='any', inplace=True)
    print('high shape after: ', adjclose_df_original.shape)    
if volume_df_original.isnull().values.any():
    print('high shape before: ', volume_df_original.shape)
    volume_df_original.dropna(axis=1, how='any', inplace=True)
    print('high shape after: ', volume_df_original.shape)   

open shape before:  (415, 4541)
open shape after:  (415, 4534)
high shape before:  (415, 4541)
high shape after:  (415, 4534)
low shape before:  (415, 4541)
low shape after:  (415, 4534)
high shape before:  (415, 4541)
high shape after:  (415, 4534)
high shape before:  (415, 4541)
high shape after:  (415, 4534)


In [5]:
open_df = open_df_original.drop(index=open_df_original.iloc[-1:].index)
high_df = high_df_original.drop(index=high_df_original.iloc[-1:].index)
low_df = low_df_original.drop(index=low_df_original.iloc[-1:].index)
adjclose_df = adjclose_df_original.drop(index=adjclose_df_original.iloc[-1:].index)
volume_df = volume_df_original.drop(index=volume_df_original.iloc[-1:].index)

In [ ]:
open_df

,A,AA,AACG,AAIC,AAL,AAMC,AAME,AAOI,AAON,AAP,AAPL,AAU,AAWW,AAXN,ABB,ABBV,ABC,ABCB,ABEO,ABEV,ABG,ABIO,ABM,ABMD,ABR-PA,ABR-PB,ABR-PC,ABR,ABT,ABTX,ABUS,AC,ACA,ACAD,ACB,ACBI,ACC,ACCO,ACER,ACET,...,YNDX,YORW,YPF,YRCW,YRD,YTEN,YTRA,YUM,YUMC,YVR,YY,Z,ZAGG,ZBH,ZBRA,ZDGE,ZEAL,ZEN,ZEUS,ZG,ZGNX,ZION,ZIOP,ZIXI,ZKIN,ZLAB,ZM,ZNGA,ZNH,ZOM,ZS,ZSAN,ZTO,ZTS,ZUMZ,ZUO,ZVO,ZYME,ZYNE,ZYXI
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-07-29,69.500000,23.059999,2.61,6.59,30.969999,10.550000,2.49,9.28,51.639999,152.289993,52.115002,0.75,45.560001,69.680000,18.559999,67.870003,86.110001,39.700001,3.17,5.23,88.760002,6.95,42.000000,271.980011,26.100000,25.8543,26.508200,12.11,88.059998,35.029999,1.85,36.746269,36.369999,23.450001,76.320000,18.209999,46.980000,8.09,2.78,66.919998,...,38.740002,35.689999,16.639999,3.390,11.44,34.400002,4.500,113.669998,45.130001,1.74,64.059998,49.320000,6.40,136.039993,190.250000,1.700,22.910000,94.180000,12.240000,49.070000,48.720001,45.099998,6.13,8.92,1.300,35.070000,102.599998,6.51,33.130001,0.250,88.500000,2.810,19.639999,115.070000,25.959999,15.60,3.83,24.250000,11.300,8.060000
2019-07-30,69.250000,22.900000,2.64,6.66,30.450001,10.470000,2.41,9.21,50.240002,152.520004,52.189999,0.76,44.990002,68.000000,18.770000,66.919998,86.500000,38.869999,3.01,5.42,94.000000,6.64,41.509998,273.869995,26.200001,25.8050,26.549999,12.20,88.180000,33.759998,1.75,36.955223,35.500000,23.990000,72.720001,18.160000,46.750000,8.10,2.74,73.849998,...,39.700001,35.689999,16.770000,3.150,11.15,34.799999,4.520,113.860001,44.959999,1.65,63.980000,48.610001,6.42,134.000000,206.449997,1.575,22.940001,91.699997,11.920000,48.650002,47.930000,44.400002,5.95,8.68,1.290,35.490002,95.190002,6.37,32.830002,0.255,84.699997,2.820,19.940001,115.000000,25.440001,15.06,3.87,23.980000,10.640,8.280000
2019-07-31,70.190002,23.049999,2.61,6.72,30.680000,9.990000,2.48,9.54,51.779999,151.649994,54.105000,0.75,46.389999,69.760002,19.049999,66.910004,88.720001,40.000000,2.97,5.32,93.000000,6.80,42.330002,280.369995,26.337000,25.8102,26.503500,12.27,88.180000,33.669998,1.91,37.144279,37.220001,25.230000,76.440002,18.410000,46.919998,8.90,2.89,77.629997,...,39.000000,36.099998,16.700001,3.170,11.02,34.799999,4.550,113.290001,45.299999,1.65,63.939999,48.900002,6.65,136.139999,215.559998,1.862,22.500000,86.699997,12.270000,48.759998,49.080002,45.400002,6.64,8.84,1.290,33.299999,96.430000,6.45,32.570000,0.257,85.199997,2.850,19.980000,115.470001,24.930000,15.53,3.90,23.150000,10.940,8.480000
2019-08-01,69.629997,22.270000,2.64,6.54,30.430000,10.040000,2.35,10.02,49.110001,150.520004,53.474998,0.75,40.689999,70.120003,18.660000,66.930000,90.879997,38.450001,2.59,5.28,92.860001,6.90,42.099998,222.699997,26.389999,25.8102,26.429600,12.18,87.180000,33.480000,1.79,37.333332,37.320000,26.090000,74.639999,18.350000,46.750000,9.78,3.03,77.000000,...,38.970001,36.119999,16.260000,3.230,11.15,34.000000,4.580,116.150002,45.840000,1.74,64.430000,50.000000,6.61,135.199997,211.710007,1.620,22.587999,83.820000,12.880000,49.959999,47.980000,44.849998,7.02,9.09,1.290,31.750000,95.750000,6.48,32.169998,0.250,84.830002,2.900,19.650000,114.900002,24.860001,15.06,3.80,22.879999,10.590,8.940000
2019-08-02,69.120003,21.150000,2.60,6.39,29.700001,8.620000,2.35,9.94,51.180000,149.600006,51.382500,0.76,34.610001,68.089996,18.620001,65.500000,91.949997,38.459999,2.52,5.28,91.199997,6.79,40.950001,197.970001,26.330000,25.9000,26.780001,12.42,85.949997,32.389999,1.72,36.975124,36.000000,28.629999,74.040001,17.830000,46.380001,9.69,2.72,74.550003,...,38.290001,35.799999,16.260000,3.010,10.70,32.799999,4.500,116.709999,44.619999,1.69,60.599998,49.040001,6.35,137.229996,201.929993,1.620,22.330000,82.089996,11.900000,49.240002,48.400002,42.849998,6.74,9.89,1.200,31.940001,94.919998,6.35,31.100000,0.261,84.139999,2.770,19.250000,115.699997,22.799999,14.78,3.85,23.250000,10.450,9.110000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

# 2. Second part: computation and assessing 

### &nbsp;&nbsp;&nbsp; * SKLearn preprocessing import to scale data

In [45]:
from sklearn import preprocessing

### &nbsp;&nbsp;&nbsp; %FEATURE% Relative variation from open to adjusted close price

In [51]:
# relative variation from open to adjusted close price
adjclose_rel_var_df = (adjclose_df-open_df)/open_df

### &nbsp;&nbsp;&nbsp; %FEATURE% Absolute variation between high and low price

In [52]:
# absolute variation between high and low price
high_low_var_df = (high_df-low_df)

# to scale the absolute variation between min&max value
min0_max1_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
high_low_var_scaled = min0_max1_scaler.fit_transform(high_low_var_df)
high_low_var_scaled_df = pd.DataFrame(data=high_low_var_scaled, index=high_low_var_df.index, columns=high_low_var_df.columns)

### &nbsp;&nbsp;&nbsp; %FEATURE% High low absolute variation over adjusted close price

In [53]:
# high low absolute variation over adjusted close price
high_low_var_df_adjclose = high_low_var_df/adjclose_df

### &nbsp;&nbsp;&nbsp; %FEATURE% Log Return (adjusted close price log return)

In [54]:
adjclose_df_log_return = np.log(adjclose_df/adjclose_df.shift(1))

In [55]:
adjclose_rel_var_df = adjclose_rel_var_df.transpose()
high_low_var_scaled_df = high_low_var_scaled_df.transpose()

In [82]:
for num, col in enumerate(adjclose_rel_var_df.columns):
  lista = [np.array([col, col]), np.array(['Price','Index'])] 
  if num == 0:
    df_trial = pd.DataFrame(data=np.column_stack((adjclose_rel_var_df[col].values, high_low_var_scaled_df[col].values)), index= join.index, columns=lista)
  else:
    df_trial2 = pd.DataFrame(data=np.column_stack((adjclose_rel_var_df[col].values, high_low_var_scaled_df[col].values)), index= join.index, columns=lista)
    df_trial = df_trial.join(df_trial2, how='left')

In [100]:
last_df = df_trial.stack(list(range(df_trial.columns.nlevels)))
last_df2 = last_df

A     2019-07-29  Index    0.092069
                  Price    0.000102
      2019-07-30  Index    0.228046
                  Price    0.000575
      2019-07-31  Index    0.179888
                             ...   
ZYXI  2021-03-16  Price   -0.019512
      2021-03-17  Index    0.138947
                  Price    0.020664
      2021-03-18  Index    0.187368
                  Price   -0.049140
Length: 3754152, dtype: float64

In [102]:
data = {'open': last_df, 
        'high': last_df2}

df_FinaleConcatenato = pd.concat(data, axis=1)
print(df_FinaleConcatenato.shape)
df_FinaleConcatenato.dropna(axis = 0, how = 'any', inplace=True)
print(df_FinaleConcatenato.shape)

(3754152, 2)
(3754152, 2)


In [107]:
df_FinaleConcatenato.loc['A', '2019-07-29']

,open,high
Index,0.092069,0.092069
Price,0.000102,0.000102


In [86]:
index = pd.MultiIndex.from_product([['a', 'b'], ['A', 'B'], ['One', 'Two']])
df = pd.DataFrame(np.arange(16).reshape(2, 8), columns=index)
df

a               b            
    A       B       A       B    
  One Two One Two One Two One Two
0   0   1   2   3   4   5   6   7
1   8   9  10  11  12  13  14  15

In [88]:
print (df.columns.nlevels)

print (list(range(df.columns.nlevels)))

print (df.stack(list(range(df.columns.nlevels))))

3
[0, 1, 2]
0  a  A  One     0
         Two     1
      B  One     2
         Two     3
   b  A  One     4
         Two     5
      B  One     6
         Two     7
1  a  A  One     8
         Two     9
      B  One    10
         Two    11
   b  A  One    12
         Two    13
      B  One    14
         Two    15
dtype: int64


In [85]:
df_trial.stack(dropna=False)

2019-07-29  2019-07-30  ...  2021-03-17  2021-03-18
A    Index    0.092069    0.228046  ...    0.161474    0.189803
     Price    0.000102    0.000575  ...    0.001471   -0.008172
AA   Index    0.147627    0.075571  ...    0.534270    0.797891
     Price    0.002602    0.006114  ...    0.054359   -0.014845
AACG Index    0.003067    0.004908  ...    0.025767    0.041718
...                ...         ...  ...         ...         ...
ZYME Price   -0.003711   -0.032944  ...   -0.022255   -0.010255
ZYNE Index    0.401050    0.174288  ...    0.116192    0.200525
     Price   -0.045133    0.016917  ...    0.057372   -0.103896
ZYXI Index    0.029474    0.042105  ...    0.138947    0.187368
     Price    0.028536    0.013285  ...    0.020664   -0.049140

[9068 rows x 414 columns]

In [69]:
adjclose_rel_var_df[col].values

array([ 0.0001022 ,  0.00260197, -0.00383141, ..., -0.00371135,
       -0.04513276,  0.02853592])

In [77]:
pp=np.column_stack((adjclose_rel_var_df[col].values, adjclose_rel_var_df[col].values))
pd.DataFrame(data=pp, index= join.index, columns=lista)

# pd.DataFrame(data=pp).transpose().values

2019-07-29          
          Price     Index
A      0.000102  0.000102
AA     0.002602  0.002602
AACG  -0.003831 -0.003831
AAIC  -0.063212 -0.063212
AAL   -0.017731 -0.017731
...         ...       ...
ZUO   -0.026282 -0.026282
ZVO   -0.005222 -0.005222
ZYME  -0.003711 -0.003711
ZYNE  -0.045133 -0.045133
ZYXI   0.028536  0.028536

[4534 rows x 2 columns]

In [58]:
col = '2019-07-29'
adjclose_df_log_return[col].values

array([ 0.0001022 ,  0.00260197, -0.00383141, ..., -0.00371135,
       -0.04513276,  0.02853592])

### &nbsp;&nbsp;&nbsp; * Construction of stack dataset with all features and label to classify

In [50]:
adjclose_rel_var_df = adjclose_rel_var_df.transpose()
high_low_var_scaled_df = high_low_var_scaled_df.transpose()

In [12]:
colonne = pd.MultiIndex.from_product([adjclose_rel_var_df.columns, ['Price']])
adjclose_rel_var_df=pd.DataFrame(adjclose_rel_var_df.values, columns=colonne, index=adjclose_rel_var_df.index)
colonne = pd.MultiIndex.from_product([high_low_var_scaled_df.columns, ['Index']])
high_low_var_scaled_df=pd.DataFrame(high_low_var_scaled_df.values, columns=colonne, index=high_low_var_scaled_df.index)

In [84]:
join=pd.DataFrame(adjclose_rel_var_df.join(high_low_var_scaled_df), index=adjclose_rel_var_df.index)

ValueError: ignored

In [44]:

lista = [np.array(['2019-07-29', '2019-07-29']), np.array(['Price','Index'])] 
lista
pd.DataFrame(data=join['2019-07-29'].values, index= join.index, columns=lista)

2019-07-29          
          Price     Index
A      0.000102  0.092069
AA     0.002602  0.147627
AACG  -0.003831  0.003067
AAIC  -0.063212  0.052941
AAL   -0.017731  0.078947
...         ...       ...
ZUO   -0.026282  0.287705
ZVO   -0.005222  0.120805
ZYME  -0.003711  0.024611
ZYNE  -0.045133  0.401050
ZYXI   0.028536  0.029474

[4534 rows x 2 columns]

In [16]:
join['2019-07-29']pd.DataFrame(adjclose_rel_var_df.merge(high_low_var_scaled_df), index=adjclose_rel_var_df.index)

MergeError: ignored

In [15]:
join['2019-07-29']

,Price,Index
A,0.000102,0.092069
AA,0.002602,0.147627
AACG,-0.003831,0.003067
AAIC,-0.063212,0.052941
AAL,-0.017731,0.078947
...,...,...
ZUO,-0.026282,0.287705
ZVO,-0.005222,0.120805
ZYME,-0.003711,0.024611
ZYNE,-0.045133,0.401050


In [127]:
join.stack(dropna=False)

date        2019-07-29  2019-07-30  ...  2021-03-17  2021-03-18
A    Index    0.092069    0.228046  ...    0.161474    0.189803
     Price    0.000102    0.000575  ...    0.001471   -0.008172
AA   Index    0.147627    0.075571  ...    0.534270    0.797891
     Price    0.002602    0.006114  ...    0.054359   -0.014845
AACG Index    0.003067    0.004908  ...    0.025767    0.041718
...                ...         ...  ...         ...         ...
ZYME Price   -0.003711   -0.032944  ...   -0.022255   -0.010255
ZYNE Index    0.401050    0.174288  ...    0.116192    0.200525
     Price   -0.045133    0.016917  ...    0.057372   -0.103896
ZYXI Index    0.029474    0.042105  ...    0.138947    0.187368
     Price    0.028536    0.013285  ...    0.020664   -0.049140

[9068 rows x 414 columns]

In [123]:
join=pd.DataFrame(adjclose_rel_var_df.join(high_low_var_scaled_df), index=adjclose_rel_var_df.index)

In [125]:
join_t = join.transpose()

In [126]:
join_stack=join_t.stack(dropna=False)
join_stack

date                   
2019-07-29  Price  A       0.000102
                   AA      0.002602
                   AACG   -0.003831
                   AAIC   -0.063212
                   AAL    -0.017731
                             ...   
2021-03-18  Index  ZUO     0.221312
                   ZVO     0.201342
                   ZYME    0.176620
                   ZYNE    0.200525
                   ZYXI    0.187368
Length: 3754152, dtype: float64

In [21]:
adjclose_rel_var_df_t = adjclose_rel_var_df.transpose()
adjclose_rel_var_df_stack = adjclose_rel_var_df_t.stack(dropna=False)

high_low_var_scaled_df_t = high_low_var_scaled_df.transpose()
high_low_var_scaled_df_stack = high_low_var_scaled_df_t.stack(dropna=False)

high_low_var_df_adjclose_t = high_low_var_df_adjclose.transpose()
high_low_var_df_adjclose_stack = high_low_var_df_adjclose_t.stack(dropna=False)

# to scale the adjclose between min&max value
adjclose_scaled = min0_max1_scaler.fit_transform(adjclose_df)
adjclose_scaled_df = pd.DataFrame(data=adjclose_scaled, index=adjclose_df.index, columns=adjclose_df.columns)
adjclose_scaled_df_t = adjclose_scaled_df.transpose()
adjclose_scaled_df_stack = adjclose_scaled_df_t.stack(dropna=False)

# to scale the volume between min&max value
volume_scaled = min0_max1_scaler.fit_transform(volume_df)
volume_scaled_df = pd.DataFrame(data=volume_scaled, index=volume_df.index, columns=volume_df.columns)
volume_scaled_df_t = volume_scaled_df.transpose()
volume_scaled_df_stack = volume_scaled_df_t.stack(dropna=False)

# classification variable 
classification_df = (adjclose_df_log_return > 0) * 1
classification_df_t = classification_df.transpose()
classification_df_t.shift(axis = 1, periods = -1) # sign the day before a rally up with 1 and vice versa with 0 
classification_df_stack = classification_df_t.shift(axis = 1, periods = -1).stack(dropna=False)

data = {'adjclose_rel_var': adjclose_rel_var_df_stack, 
        'high_low_var_scaled': high_low_var_scaled_df_stack,
        'high_low_var_adjclose': high_low_var_df_adjclose_stack,
        'adjclose_scaled': adjclose_scaled_df_stack,
        'volume_scaled': volume_scaled_df_stack,
        'label': classification_df_stack}  

df_concat = pd.concat(data, axis=1)
df_concat.dropna(axis = 0, how = 'any', inplace=True)
df_concat

adjclose_rel_var  high_low_var_scaled  ...  volume_scaled  label
     date                                               ...                      
A    2019-07-29          0.000102             0.092069  ...       0.385451    0.0
     2019-07-30          0.000575             0.228046  ...       0.404396    0.0
     2019-07-31         -0.023239             0.179888  ...       0.555941    0.0
     2019-08-01         -0.018220             0.257791  ...       0.345647    0.0
     2019-08-02         -0.027124             0.155808  ...       0.393772    0.0
...                           ...                  ...  ...            ...    ...
ZYXI 2021-03-11          0.009512             0.113684  ...       0.070525    0.0
     2021-03-12         -0.000592             0.044211  ...       0.054630    0.0
     2021-03-15         -0.028554             0.147369  ...       0.061491    0.0
     2021-03-16         -0.019512             0.191579  ...       0.064782    1.0
     2021-03-17          0.020664             0.138947  ...       0.024764    0.0

[1872542 rows x 6 columns]

### &nbsp;&nbsp;&nbsp; * Check the correct number of row in the previous dataframe

In [22]:
# check the correct number of row in the previous dataframe
len(open_df.columns) * len(open_df.index) - len(open_df.columns)

1872542

### &nbsp;&nbsp;&nbsp; * Take the correct list of tickers

In [23]:
ticker_list = []
for item in df_concat.index:
    ticker_list.append(item[0])
ticker_list = list(dict.fromkeys(ticker_list))

### &nbsp;&nbsp;&nbsp; * Construction of stack dataset with other features and label to classify

In [24]:
open_df_t = open_df.loc[adjclose_df_log_return.index[0]:adjclose_df_log_return.index[-1]].transpose()
open_df_stack = open_df_t.stack()

high_df_t = high_df.loc[adjclose_df_log_return.index[0]:adjclose_df_log_return.index[-1]].transpose()
high_df_stack = high_df_t.stack()

low_df_t = low_df.loc[adjclose_df_log_return.index[0]:adjclose_df_log_return.index[-1]].transpose()
low_df_stack = low_df_t.stack()

adjclose_df_t = adjclose_scaled_df.transpose()
adjclose_df_stack = adjclose_scaled_df_t.stack(dropna=False)

volume_df_t = volume_df.loc[adjclose_df_log_return.index[0]:adjclose_df_log_return.index[-1]].transpose()
volume_df_stack = volume_df_t.stack()

data = {'open': open_df_stack, 
        'high': high_df_stack,
        'low': low_df_stack,
        'adjclose': adjclose_df_stack,
        'volume': volume_df_stack,
        'label': classification_df_stack} # taken from before computation

df_concat2 = pd.concat(data, axis = 1)
df_concat2.dropna(axis = 0, how = 'any', inplace = True)
df_concat2

open       high        low  adjclose     volume  label
     date                                                                   
A    2019-07-29  69.500000  70.500000  69.379997  0.098826  2750000.0    0.0
     2019-07-30  69.250000  70.180000  68.099998  0.095595  2871800.0    0.0
     2019-07-31  70.190002  70.660004  68.919998  0.084728  3846100.0    0.0
     2019-08-01  69.629997  70.680000  68.389999  0.081791  2494100.0    0.0
     2019-08-02  69.120003  69.129997  67.559998  0.065196  2803500.0    0.0
...                    ...        ...        ...       ...        ...    ...
ZYXI 2021-03-11  16.820000  17.110001  16.370001  0.446301   518100.0    0.0
     2021-03-12  16.879999  16.959999  16.549999  0.441050   421500.0    0.0
     2021-03-15  16.809999  17.030001  16.129999  0.415274   463200.0    0.0
     2021-03-16  16.400000  16.709999  15.600000  0.403341   483200.0    1.0
     2021-03-17  15.970000  16.490000  15.630000  0.413842   240000.0    0.0

[1872542 rows x 6 columns]

##### Remember: .std() is sample deviation, whereas the standardization thru StandardScaler use the standard deviation (the difference is the denominator, in the sample std it is used N-1). Moreover, to compute the standard deviation, you can do it in this way: .std(ddof=0)

# 3. Third part: logistic regression 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

### &nbsp;&nbsp;&nbsp; %FUNCTION% Function for logistic regression

In [ ]:
def logistic_reg(features_df, classification_array, train_size = 0.80, shuffle_value = True, stratify_value = None, random_state=None):
    
    x_train, x_test, y_train, y_test = train_test_split(features_df, classification_array, train_size = train_size, shuffle = shuffle_value, stratify = stratify_value, random_state = random_state)

    lr = LogisticRegression()

    lr.fit(x_train, y_train)

    lr_pred = lr.predict(x_test)

    target_names = ['class 0', 'class 1']

    return(confusion_matrix(y_test, lr_pred), classification_report(y_test, lr_pred, target_names=target_names, output_dict=True), lr)

### &nbsp;&nbsp;&nbsp; * Definition of the StockClass dictionary

In [ ]:
# Definition of a dictionary to store stock as StockClass instances and for each stock get attribute
# (take a look at _2_0_stock_dataframe_class.py for more information)
stock_object_dictionary = {'{0}'.format(ticker): StockClass(ticker=ticker_list) for ticker in ticker_list}

### a. Trial 1: No Shuffle (i.e. No Stratify) - 80/20 - scaled features

In [ ]:
%%time
for ticker in ticker_list:
    
    try:
        # features dataframe, without classification column
        features_df = df_concat.loc[ticker]
        features_df.drop('label', axis=1, inplace=True)
        
        # classification array
        classification_array = df_concat.loc[ticker]['label'].values

        # sacaling data
        df_sc_scaled = StandardScaler().fit_transform(features_df)

        shuffle_value = False # no shuffle, because of this we cannot stratify our label
        stratify_value = None
        train_size = 0.80
        random_state = None

        stock_object_dictionary['{0}'.format(ticker)].confusion_matrix, stock_object_dictionary['{0}'.format(ticker)].accuracy_report, stock_object_dictionary['{0}'.format(ticker)].logistic_regression = logistic_reg(df_sc_scaled, classification_array, train_size, shuffle_value, stratify_value, random_state) 

    except:
        pass

CPU times: user 1min 49s, sys: 1.28 s, total: 1min 51s
Wall time: 1min 51s


### b. Trial 2: Shuffle&Stratify - 80/20 - scaled features

In [ ]:
%%time
for ticker in ticker_list:
    
    try:
        # features dataframe, without classification column
        features_df = df_concat.loc[ticker]
        features_df.drop('label', axis=1, inplace=True)
        
        # classification array
        classification_array = df_concat.loc[ticker]['label'].values

        # sacaling data
        df_sc_scaled = StandardScaler().fit_transform(features_df)

        shuffle_value = True # shuffle, because of this we cannot stratify our label
        stratify_value = classification_array # stratify fashion 
        train_size = 0.80
        random_state = None

        stock_object_dictionary['{0}'.format(ticker)].confusion_matrix2, stock_object_dictionary['{0}'.format(ticker)].accuracy_report2, stock_object_dictionary['{0}'.format(ticker)].logistic_regression2 = logistic_reg(df_sc_scaled, classification_array, train_size, shuffle_value, stratify_value, random_state) 
    
    except:
        pass

CPU times: user 1min 50s, sys: 1.24 s, total: 1min 51s
Wall time: 1min 52s


In [ ]:
%%time
for j, ticker in enumerate(ticker_list):
  
  try:
      column_list = []
      vector_values = [] 
      data = pd.DataFrame(data=stock_object_dictionary['{0}'.format(ticker)].accuracy_report)

      for c in data.columns:
        for i in data.index:
          if c != 'accuracy':
            if c != 'macro avg' and c != 'weighted avg':
              column_list.append(str(c + '-' + i))
              vector_values.append(data[c][i])
            else:
              if i != 'support':
                column_list.append(str(c + '-' + i))
                vector_values.append(data[c][i])
          elif i=='precision':
            column_list.append(str(c))
            vector_values.append(data[c][i])

      if j == 0:
        matrix=np.array(vector_values).reshape(1,len(np.array(vector_values)))
      else:
        vector_values = np.array(vector_values).reshape(1,len(np.array(vector_values)))
        matrix=np.concatenate((matrix, vector_values))
         
  except:
    pass      

CPU times: user 5.52 s, sys: 26 ms, total: 5.55 s
Wall time: 5.55 s


In [ ]:
matrix

array([[0.6       , 0.14634146, 0.23529412, ..., 0.55979535, 0.53012048,
        0.45064555],
       [0.46987952, 1.        , 0.63934426, ..., 0.22078676, 0.46987952,
        0.30041477],
       [0.55555556, 0.97826087, 0.70866142, ..., 0.53078983, 0.55421687,
        0.41561278],
       ...,
       [0.60416667, 0.63043478, 0.61702128, ..., 0.56409925, 0.56626506,
        0.56485517],
       [0.56164384, 0.93181818, 0.7008547 , ..., 0.62665456, 0.57831325,
        0.50578872],
       [0.55737705, 0.79069767, 0.65384615, ..., 0.57353707, 0.56626506,
        0.54083829]])

In [ ]:
stock_report_df=pd.DataFrame(data=np.array(matrix), columns=column_list, index=ticker_list)
stock_report_df

,class 0-precision,class 0-recall,class 0-f1-score,class 0-support,class 1-precision,class 1-recall,class 1-f1-score,class 1-support,accuracy,macro avg-precision,macro avg-recall,macro avg-f1-score,weighted avg-precision,weighted avg-recall,weighted avg-f1-score
A,0.600000,0.146341,0.235294,41.0,0.520548,0.904762,0.660870,42.0,0.530120,0.560274,0.525552,0.448082,0.559795,0.530120,0.450646
AA,0.469880,1.000000,0.639344,39.0,0.000000,0.000000,0.000000,44.0,0.469880,0.234940,0.500000,0.319672,0.220787,0.469880,0.300415
AACG,0.555556,0.978261,0.708661,46.0,0.500000,0.027027,0.051282,37.0,0.554217,0.527778,0.502644,0.379972,0.530790,0.554217,0.415613
AAIC,0.493976,1.000000,0.661290,41.0,0.000000,0.000000,0.000000,42.0,0.493976,0.246988,0.500000,0.330645,0.244012,0.493976,0.326661
AAL,0.445783,1.000000,0.616667,37.0,0.000000,0.000000,0.000000,46.0,0.445783,0.222892,0.500000,0.308333,0.198723,0.445783,0.274900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZUO,0.333333,0.361111,0.346667,36.0,0.477273,0.446809,0.461538,47.0,0.409639,0.405303,0.403960,0.404103,0.414841,0.409639,0.411715
ZVO,0.469136,0.974359,0.633333,39.0,0.500000,0.022727,0.043478,44.0,0.469880,0.484568,0.498543,0.338406,0.485498,0.469880,0.320639
ZYME,0.604167,0.630435,0.617021,46.0,0.514286,0.486486,0.500000,37.0,0.566265,0.559226,0.558461,0.558511,0.564099,0.566265,0.564855
ZYNE,0.561644,0.931818,0.700855,44.0,0.700000,0.179487,0.285714,39.0,0.578313,0.630822,0.555653,0.493284,0.626655,0.578313,0.505789


In [ ]:
support_columns=[]
for c in stock_report_df.columns:
  if 'support' in c:
    support_columns.append(str(c))
support_columns

['class 0-support', 'class 1-support']

In [ ]:
 stock_report_df['the lowest support'] = stock_report_df[support_columns].min(axis=1)/(stock_report_df[support_columns].sum().sum()/len(stock_report_df.index))

In [ ]:
stock_report_filtered_df=stock_report_df[stock_report_df['the lowest support']>=.49]
stock_report_filtered_df

,class 0-precision,class 0-recall,class 0-f1-score,class 0-support,class 1-precision,class 1-recall,class 1-f1-score,class 1-support,accuracy,macro avg-precision,macro avg-recall,macro avg-f1-score,weighted avg-precision,weighted avg-recall,weighted avg-f1-score,the lowest support
A,0.600000,0.146341,0.235294,41.0,0.520548,0.904762,0.660870,42.0,0.530120,0.560274,0.525552,0.448082,0.559795,0.530120,0.450646,0.493976
AAIC,0.493976,1.000000,0.661290,41.0,0.000000,0.000000,0.000000,42.0,0.493976,0.246988,0.500000,0.330645,0.244012,0.493976,0.326661,0.493976
ABC,0.506173,0.976190,0.666667,42.0,0.500000,0.024390,0.046512,41.0,0.506024,0.503086,0.500290,0.356589,0.503124,0.506024,0.360325,0.493976
ABEO,0.518519,0.333333,0.405797,42.0,0.500000,0.682927,0.577320,41.0,0.506024,0.509259,0.508130,0.491558,0.509371,0.506024,0.490525,0.493976
ABR-PC,0.489362,0.560976,0.522727,41.0,0.500000,0.428571,0.461538,42.0,0.493976,0.494681,0.494774,0.492133,0.494745,0.493976,0.491764,0.493976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZION,0.500000,1.000000,0.666667,41.0,1.000000,0.023810,0.046512,42.0,0.506024,0.750000,0.511905,0.356589,0.753012,0.506024,0.352853,0.493976
ZLAB,0.466667,0.341463,0.394366,41.0,0.490566,0.619048,0.547368,42.0,0.481928,0.478616,0.480256,0.470867,0.478760,0.481928,0.471789,0.493976
ZOM,0.500000,0.952381,0.655738,42.0,0.333333,0.024390,0.045455,41.0,0.493976,0.416667,0.488386,0.350596,0.417671,0.493976,0.354273,0.493976
ZSAN,0.513889,0.880952,0.649123,42.0,0.545455,0.146341,0.230769,41.0,0.518072,0.529672,0.513647,0.439946,0.529482,0.518072,0.442466,0.493976


In [ ]:
stock_report_filtered_df.sort_values(by=['accuracy'], ascending=False, inplace=True)
stock_report_filtered_df = stock_report_filtered_df[stock_report_filtered_df['accuracy']>=.6]
stock_report_filtered_df

,class 0-precision,class 0-recall,class 0-f1-score,class 0-support,class 1-precision,class 1-recall,class 1-f1-score,class 1-support,accuracy,macro avg-precision,macro avg-recall,macro avg-f1-score,weighted avg-precision,weighted avg-recall,weighted avg-f1-score,the lowest support
TWOU,0.622642,0.804878,0.702128,41.0,0.733333,0.523810,0.611111,42.0,0.662651,0.677987,0.664344,0.656619,0.678654,0.662651,0.656071,0.493976
BCML,0.651163,0.666667,0.658824,42.0,0.650000,0.634146,0.641975,41.0,0.650602,0.650581,0.650407,0.650399,0.650588,0.650602,0.650501,0.493976
RNR-PE,0.625000,0.714286,0.666667,42.0,0.657143,0.560976,0.605263,41.0,0.638554,0.641071,0.637631,0.635965,0.640878,0.638554,0.636335,0.493976
GNFT,0.600000,0.804878,0.687500,41.0,0.714286,0.476190,0.571429,42.0,0.638554,0.657143,0.640534,0.629464,0.657831,0.638554,0.628765,0.493976
YEXT,0.647059,0.536585,0.586667,41.0,0.612245,0.714286,0.659341,42.0,0.626506,0.629652,0.625436,0.623004,0.629442,0.626506,0.623441,0.493976
ACIU,0.789474,0.357143,0.491803,42.0,0.578125,0.902439,0.704762,41.0,0.626506,0.683799,0.629791,0.598283,0.685073,0.626506,0.597000,0.493976
DJCO,0.586207,0.829268,0.686869,41.0,0.720000,0.428571,0.537313,42.0,0.626506,0.653103,0.628920,0.612091,0.653909,0.626506,0.611190,0.493976
CCI,0.600000,0.658537,0.627907,41.0,0.631579,0.571429,0.600000,42.0,0.614458,0.615789,0.614983,0.613953,0.615980,0.614458,0.613785,0.493976
BDX,0.708333,0.404762,0.515152,42.0,0.576271,0.829268,0.680000,41.0,0.614458,0.642302,0.617015,0.597576,0.643098,0.614458,0.596583,0.493976
SNY,0.584906,0.756098,0.659574,41.0,0.666667,0.476190,0.555556,42.0,0.614458,0.625786,0.616144,0.607565,0.626279,0.614458,0.606938,0.493976


In [ ]:
open_filtered_df = open_df.iloc[1:][stock_report_filtered_df.index]
high_filtered_df = high_df.iloc[1:][stock_report_filtered_df.index]
low_filtered_df = low_df.iloc[1:][stock_report_filtered_df.index]
adjclose_filtered_df = adjclose_df.iloc[1:][stock_report_filtered_df.index]
volume_filtered_df = volume_df.iloc[1:][stock_report_filtered_df.index]
last_date = volume_filtered_df.index[-1]

In [ ]:
# relative variation from open to adjusted close price
adjclose_rel_var_filtered_df = (adjclose_filtered_df-open_filtered_df)/open_filtered_df

# absolute variation between high and low price
high_low_var_filtered_df = (high_filtered_df-low_filtered_df)

# to scale the absolute variation between min&max value
min0_max1_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
high_low_var_filtered_scaled = min0_max1_scaler.fit_transform(high_low_var_filtered_df)
high_low_var_filtered_scaled_df = pd.DataFrame(data=high_low_var_filtered_scaled, index=high_low_var_filtered_df.index, columns=high_low_var_filtered_df.columns)

# high low absolute variation over adjusted close price
high_low_var_filtered_df_adjclose = high_low_var_filtered_df/adjclose_filtered_df

adjclose_rel_var_filtered_df_t = adjclose_rel_var_filtered_df.transpose()
adjclose_rel_var_filtered_df_stack = adjclose_rel_var_filtered_df_t.stack(dropna=False)

high_low_var_filtered_scaled_df_t = high_low_var_filtered_scaled_df.transpose()
high_low_var_filtered_scaled_df_stack = high_low_var_filtered_scaled_df_t.stack(dropna=False)

high_low_var_filtered_df_adjclose_t = high_low_var_filtered_df_adjclose.transpose()
high_low_var_filtered_df_adjclose_stack = high_low_var_filtered_df_adjclose_t.stack(dropna=False)

# to scale the adjclose between min&max value
adjclose_filtered_scaled = min0_max1_scaler.fit_transform(adjclose_filtered_df)
adjclose_filtered_scaled_df = pd.DataFrame(data=adjclose_filtered_df, index=adjclose_filtered_df.index, columns=adjclose_filtered_df.columns)
adjclose_filtered_scaled_df_t = adjclose_filtered_scaled_df.transpose()
adjclose_filtered_scaled_df_stack = adjclose_filtered_scaled_df_t.stack(dropna=False)

# to scale the volume between min&max value
volume_filtered_scaled = min0_max1_scaler.fit_transform(volume_filtered_df)
adjclose_filtered_scaled_df = pd.DataFrame(data=volume_filtered_scaled, index=volume_filtered_df.index, columns=volume_filtered_df.columns)
volume_filtered_scaled_df_t = adjclose_filtered_scaled_df.transpose()
volume_filtered_scaled_df_stack = volume_filtered_scaled_df_t.stack(dropna=False)

data = {'adjclose_rel_var': adjclose_rel_var_filtered_df_stack, 
        'high_low_var_scaled': high_low_var_filtered_scaled_df_stack,
        'high_low_var_adjclose': high_low_var_filtered_df_adjclose_stack,
        'adjclose_scaled': adjclose_filtered_scaled_df_stack,
        'volume_scaled': volume_filtered_scaled_df_stack}  

df_filtered_concat = pd.concat(data, axis=1)
df_filtered_concat.dropna(axis = 0, how = 'any', inplace=True)
df_filtered_concat

adjclose_rel_var  ...  volume_scaled
     date                          ...               
TWOU 2019-07-30          0.059199  ...       0.028053
     2019-07-31         -0.326670  ...       1.000000
     2019-08-01          0.113600  ...       0.269027
     2019-08-02         -0.001460  ...       0.100534
     2019-08-05          0.035848  ...       0.073569
...                           ...  ...            ...
HRTG 2021-03-12         -0.004303  ...       0.234948
     2021-03-15          0.029488  ...       0.134602
     2021-03-16         -0.012658  ...       0.180104
     2021-03-17         -0.002558  ...       0.105190
     2021-03-18          0.020530  ...       0.294637

[7434 rows x 5 columns]

In [ ]:
df_prediction_concat = df_filtered_concat.loc[(df_filtered_concat.index.get_level_values('date') == last_date)]
df_prediction_concat

,,adjclose_rel_var,high_low_var_scaled,high_low_var_adjclose,adjclose_scaled,volume_scaled
,date,,,,,
TWOU,2021-03-18,-0.013194,0.228454,0.058217,35.900002,0.028293
BCML,2021-03-18,0.011068,0.146766,0.036672,18.270000,0.067376
RNR-PE,2021-03-18,-0.001591,0.011192,0.003586,25.100000,0.114549
GNFT,2021-03-18,-0.006198,0.004535,0.012474,4.810000,0.001714
YEXT,2021-03-18,-0.039769,0.193878,0.052772,14.970000,0.081604
ACIU,2021-03-18,0.000000,0.081197,0.061697,7.780000,0.004705
DJCO,2021-03-18,-0.022718,0.462162,0.051691,330.809998,0.452703
CCI,2021-03-18,0.007871,0.174985,0.021854,165.190002,0.187405
BDX,2021-03-18,0.002431,0.048154,0.012375,239.199997,0.056304


In [ ]:
prediciton=[]
for ticker, d in df_prediction_concat.index:
   prediciton.append(stock_object_dictionary['{0}'.format(ticker)].logistic_regression.predict(df_prediction_concat.loc[[ticker]].values))
df_prediction_concat['prediction'] = prediciton

In [ ]:
df_prediction_concat

,,adjclose_rel_var,high_low_var_scaled,high_low_var_adjclose,adjclose_scaled,volume_scaled,prediction
,date,,,,,,
TWOU,2021-03-18,-0.013194,0.228454,0.058217,35.900002,0.028293,[1.0]
BCML,2021-03-18,0.011068,0.146766,0.036672,18.270000,0.067376,[0.0]
RNR-PE,2021-03-18,-0.001591,0.011192,0.003586,25.100000,0.114549,[0.0]
GNFT,2021-03-18,-0.006198,0.004535,0.012474,4.810000,0.001714,[1.0]
YEXT,2021-03-18,-0.039769,0.193878,0.052772,14.970000,0.081604,[0.0]
ACIU,2021-03-18,0.000000,0.081197,0.061697,7.780000,0.004705,[0.0]
DJCO,2021-03-18,-0.022718,0.462162,0.051691,330.809998,0.452703,[0.0]
CCI,2021-03-18,0.007871,0.174985,0.021854,165.190002,0.187405,[0.0]
BDX,2021-03-18,0.002431,0.048154,0.012375,239.199997,0.056304,[0.0]


In [ ]:
open_filtered_df_t = open_filtered_df.transpose()
open_filtered_df_stack = open_filtered_df_t.stack(dropna=False)

high_filtered_df_t = high_filtered_df.transpose()
high_filtered_df_stack = high_filtered_df_t.stack(dropna=False)

low_filtered_df_t = low_filtered_df.transpose()
low_filtered_df_stack = low_filtered_df_t.stack(dropna=False)

adjclose_filtered_df_t = adjclose_filtered_df.transpose()
adjclose_filtered_df_stack = adjclose_filtered_df_t.stack(dropna=False)

volume_filtered_df_t = volume_filtered_df.transpose()
volume_filtered_df_stack = volume_filtered_df_t.stack(dropna=False)

data = {'open': open_filtered_df_stack, 
        'high': high_filtered_df_stack,
        'low': low_filtered_df_stack,
        'adjclose': adjclose_filtered_df_stack,
        'volume': volume_filtered_df_stack}  

df_filtered_concat2 = pd.concat(data, axis=1)
df_filtered_concat2.dropna(axis = 0, how = 'any', inplace=True)
df_filtered_concat2

In [ ]:
stock_report_df.sort_values(by=['accuracy'], ascending=False)

In [ ]:
for i in stock_report_df:
  print(sum(stock_report_df['class 0-support']))

### c. Trial 3: No Shuffle (i.e. No Stratify) - 70/30 - scaled features

In [ ]:
%%time
for ticker in ticker_list:
    
    try:
        # features dataframe, without classification column
        features_df = df_concat.loc[ticker]
        features_df.drop('label', axis=1, inplace=True)
        
        # classification array
        classification_array = df_concat.loc[ticker]['label'].values

        # sacaling data
        df_sc_scaled = StandardScaler().fit_transform(features_df)

        shuffle_value = False # no shuffle, because of this we cannot stratify our label
        stratify_value = None
        train_size = 0.70
        random_state = None

        stock_object_dictionary['{0}'.format(ticker)].confusion_matrix, stock_object_dictionary['{0}'.format(ticker)].accuracy_report = logistic_reg(df_sc_scaled, classification_array, train_size, shuffle_value, stratify_value, random_state) 

    except:
        pass

### d. Trial 4: Shuffle&Stratify - 70/30 - scaled features

In [ ]:
%%time
for ticker in ticker_list:
    
    try:
        # features dataframe, without classification column
        features_df = df_concat.loc[ticker]
        features_df.drop('label', axis=1, inplace=True)
        
        # classification array
        classification_array = df_concat.loc[ticker]['label'].values

        # sacaling data
        df_sc_scaled = StandardScaler().fit_transform(features_df)

        shuffle_value = True # shuffle, because of this we cannot stratify our label
        stratify_value = classification_array # stratify fashion 
        train_size = 0.70
        random_state = None

        stock_object_dictionary['{0}'.format(ticker)].confusion_matrix2, stock_object_dictionary['{0}'.format(ticker)].accuracy_report2 = logistic_reg(df_sc_scaled, classification_array, train_size, shuffle_value, stratify_value, random_state) 
    
    except:
        pass

In [ ]:
stock = 'ZYXI'

print('')
print('======================================================')
print('================   Confusion Matrix   ================')
print('======================================================')
print('')

# confusion matrix
print(stock_object_dictionary['{0}'.format(stock)].confusion_matrix)

print('')
print('======================================================')
print('=====================   Report   =====================')
print('======================================================')
print('')

# classification report
print(stock_object_dictionary['{0}'.format(stock)].accuracy_report)



print('')
print('======================================================')
print('================   Confusion Matrix   ================')
print('======================================================')
print('')

# confusion matrix
print(stock_object_dictionary['{0}'.format(stock)].confusion_matrix2)

print('')
print('======================================================')
print('=====================   Report   =====================')
print('======================================================')
print('')

# classification report
print(stock_object_dictionary['{0}'.format(stock)].accuracy_report2)